In [1]:
import pandas as pd

In [2]:
def csv_to_df(file):
    return pd.read_csv(
        file,
        parse_dates=["time"],
        usecols=[
            "id",
            "time",
            "latitude",
            "longitude",
            "mag",
            "magType",
            "place",
            "type",
            "status",
            "locationSource",
            "state",
        ],
    )

In [3]:
jan = csv_to_df("datasets/jan-enriched.csv")
feb = csv_to_df("datasets/feb-enriched.csv")
mar = csv_to_df("datasets/mar-enriched.csv")

In [4]:
sum(size for (size, _) in (jan.shape, feb.shape, mar.shape))

35919

In [5]:
earthquakes = pd.concat([jan, feb, mar])

In [6]:
earthquakes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35919 entries, 0 to 12312
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   id              35919 non-null  object             
 1   time            35919 non-null  datetime64[ns, UTC]
 2   latitude        35919 non-null  float64            
 3   longitude       35919 non-null  float64            
 4   mag             35916 non-null  float64            
 5   magType         35916 non-null  object             
 6   place           35784 non-null  object             
 7   type            35919 non-null  object             
 8   status          35919 non-null  object             
 9   locationSource  35919 non-null  object             
 10  state           31375 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(3), object(7)
memory usage: 3.3+ MB


In [7]:
earthquakes.nunique()

id                35919
time              35914
latitude          28270
longitude         29491
mag                 635
magType              10
place             16335
type                  7
status                2
locationSource       15
state               346
dtype: int64

In [8]:
earthquakes["state"].unique()

array(['US-OK', 'US-AK', 'US-CA', 'IR-18', 'ID-MA', 'US-HI', 'US-PR', nan,
       'US-WA', 'CA-YT', 'CA-BC', 'US-MO', 'US-NV', 'ID-SN', 'US-UT',
       'ID-NT', 'PG-WPD', 'US-ID', 'US-TX', 'AR-A', 'US-WY', 'FJ-E',
       'CL-AT', 'PG-ESW', 'ID-SA', 'AF-BDS', 'VU-PAM', 'ID-AC', 'JP-26',
       'DO-10', 'PG-EBR', 'ID-MU', 'US-MT', 'US-TN', 'US-KS', 'JP-02',
       'PE-UCA', 'TO-02', 'PE-HUC', 'ID-PA', 'BO-P', 'HT-NI', 'MX-CHP',
       'AR-K', 'PK-GB', 'US-OH', 'CN-SC', 'CL-RM', 'ID-BE', 'US-OR',
       'SB-TE', 'US-MN', 'AF-TAK', 'CL-AP', 'RU-SAK', 'JP-23', 'MX-BCN',
       'CL-AN', 'MX-JAL', 'JP-04', 'CL-TA', 'US-SC', 'PG-MPL', 'PG-WBK',
       'CN-XJ', 'US-CT', 'JP-30', 'CR-A', 'US-CO', 'GL-KU', 'CA-NB',
       'PG-NIK', 'AR-M', 'JP-08', 'PE-ICA', 'CN-QH', 'PH-BTG', 'DO-14',
       'CO-SAN', 'CD-TA', 'AR-J', 'US-NM', 'HT-OU', 'TW-ILA', 'PG-MRL',
       'CA-QC', 'PH-AKL', 'TO-03', 'CN-XZ', 'TJ-RA', 'AU-WA', 'ID-NB',
       'IN-UT', 'PE-ARE', 'US-MP', 'ID-JI', 'MA-03', 'DO-11', 'IR-03',


In [9]:
# We see states for countries other than the USA. Also, the states field reports null for islands and stuff.
#So dropping countries other than the USA.
us_earthquakes = earthquakes[earthquakes["state"].str.startswith("US", na=False)]

In [10]:
us_earthquakes.nunique()

id                29360
time              29355
latitude          21909
longitude         23051
mag                 578
magType               8
place             12013
type                  6
status                2
locationSource       15
state                36
dtype: int64

In [11]:
us_earthquakes[us_earthquakes["mag"].isnull()]

,id,time,latitude,longitude,mag,magType,place,type,status,locationSource,state
5013,nc71127084,2022-01-18 14:50:55.480000+00:00,35.975333,-120.371000,NaN,NaN,"10km NNE of Parkfield, CA",earthquake,reviewed,nc,US-CA
10631,nc73674341,2022-01-06 06:11:48.790000+00:00,38.798500,-122.701500,NaN,NaN,"3km NNW of Anderson Springs, CA",earthquake,reviewed,nc,US-CA
2422,pr71336423,2022-02-21 21:01:10.020000+00:00,17.956500,-66.928167,NaN,NaN,"2 km SW of Guánica, Puerto Rico",earthquake,reviewed,pr,US-PR


In [12]:
# Dropping earthquakes with missing magnitude values
us_earthquakes = us_earthquakes.dropna(subset=["mag"])

In [13]:
# Extracting state code from state data and month from time.
us_earthquakes = us_earthquakes.assign(
    state_code=us_earthquakes["state"].str.split("-").str.get(-1).str.strip(),
    month=us_earthquakes["time"].dt.month_name()
)

In [14]:
us_earthquakes.head()

,id,time,latitude,longitude,mag,magType,place,type,status,locationSource,state,state_code,month
0,ok2022ccvb,2022-01-30 23:58:39.500000+00:00,34.879167,-97.8520,0.99,ml,"7 km WSW of Alex, Oklahoma",earthquake,reviewed,ok,US-OK,OK,January
1,ak0221e05fgq,2022-01-30 23:56:19.237000+00:00,62.910000,-151.1774,1.40,ml,Central Alaska,earthquake,reviewed,ak,US-AK,AK,January
2,ak0221e051u3,2022-01-30 23:54:44.477000+00:00,63.304800,-151.2721,1.10,ml,"34 km SE of Denali National Park, Alaska",earthquake,reviewed,ak,US-AK,AK,January
3,ak0221e050xt,2022-01-30 23:54:32.973000+00:00,51.557800,-176.6738,1.80,ml,"35 km S of Adak, Alaska",earthquake,reviewed,ak,US-AK,AK,January
4,ak0221e04enn,2022-01-30 23:51:44.207000+00:00,63.291700,-151.3217,1.10,ml,"34 km SE of Denali National Park, Alaska",earthquake,reviewed,ak,US-AK,AK,January


In [15]:
us_earthquakes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29357 entries, 0 to 12311
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   id              29357 non-null  object             
 1   time            29357 non-null  datetime64[ns, UTC]
 2   latitude        29357 non-null  float64            
 3   longitude       29357 non-null  float64            
 4   mag             29357 non-null  float64            
 5   magType         29357 non-null  object             
 6   place           29281 non-null  object             
 7   type            29357 non-null  object             
 8   status          29357 non-null  object             
 9   locationSource  29357 non-null  object             
 10  state           29357 non-null  object             
 11  state_code      29357 non-null  object             
 12  month           29357 non-null  object             
dtypes: datetime64[ns, UTC](1), floa

In [16]:
# Converting values to appropriate type

In [17]:
us_earthquakes.nunique()

id                29357
time              29352
latitude          21907
longitude         23048
mag                 578
magType               8
place             12011
type                  6
status                2
locationSource       15
state                36
state_code           36
month                 3
dtype: int64

In [18]:
us_earthquakes["magType"] = us_earthquakes["magType"].astype("category")
us_earthquakes["type"] = us_earthquakes["type"].astype("category")
us_earthquakes["status"] = us_earthquakes["status"].astype("category")
us_earthquakes["locationSource"] = us_earthquakes["locationSource"].astype("category")
us_earthquakes["state"] = us_earthquakes["state"].astype("category")
us_earthquakes["state_code"] = us_earthquakes["state_code"].astype("category")
us_earthquakes["month"] = us_earthquakes["month"].astype("category")

In [19]:
us_earthquakes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29357 entries, 0 to 12311
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   id              29357 non-null  object             
 1   time            29357 non-null  datetime64[ns, UTC]
 2   latitude        29357 non-null  float64            
 3   longitude       29357 non-null  float64            
 4   mag             29357 non-null  float64            
 5   magType         29357 non-null  category           
 6   place           29281 non-null  object             
 7   type            29357 non-null  category           
 8   status          29357 non-null  category           
 9   locationSource  29357 non-null  category           
 10  state           29357 non-null  category           
 11  state_code      29357 non-null  category           
 12  month           29357 non-null  category           
dtypes: category(7), datetime64[ns, 

In [20]:
us_earthquakes.sample(5)

,id,time,latitude,longitude,mag,magType,place,type,status,locationSource,state,state_code,month
5951,hv72876857,2022-01-16 13:11:37.320000+00:00,19.148333,-155.449661,1.94,md,"6 km SSE of P?hala, Hawaii",earthquake,automatic,hv,US-HI,HI,January
4386,ak0223mulo5p,2022-03-20 12:00:25.216000+00:00,59.522200,-152.886100,1.90,ml,Southern Alaska,earthquake,reviewed,ak,US-AK,AK,March
1628,ak0223ycpgdm,2022-03-27 07:54:00.100000+00:00,65.501500,-144.236200,0.90,ml,"27 km ESE of Central, Alaska",earthquake,reviewed,ak,US-AK,AK,March
5167,uu60477922,2022-01-18 06:49:20.120000+00:00,39.424167,-110.308333,1.66,ml,"15 km SSE of Sunnyside, Utah",earthquake,reviewed,uu,US-UT,UT,January
2464,ak0223uz51qm,2022-03-25 03:35:43.820000+00:00,52.357800,-174.200200,2.40,ml,"17 km N of Atka, Alaska",earthquake,reviewed,ak,US-AK,AK,March


In [21]:
us_earthquakes.to_csv("datasets/us_earthquakes.csv", index=False)